## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lavrentiya,RU,65.5833,-171.0000,23.27,71,16,3.83,few clouds
1,1,Aklavik,CA,68.2191,-135.0107,34.52,93,90,12.66,overcast clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,31.69,98,100,7.43,light snow
3,3,Saskylakh,RU,71.9167,114.0833,16.77,93,98,10.45,overcast clouds
4,4,Buraydah,SA,26.3260,43.9750,90.05,12,2,9.62,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Mahebourg,MU,-20.4081,57.7000,72.25,88,75,13.80,broken clouds
7,7,Bredasdorp,ZA,-34.5322,20.0403,50.29,87,13,0.00,few clouds
8,8,Kalmunai,LK,7.4167,81.8167,80.49,81,100,5.46,overcast clouds
9,9,Hilo,US,19.7297,-155.0900,73.62,84,90,4.61,light rain
12,12,Nouadhibou,MR,20.9310,-17.0347,73.38,78,0,17.27,clear sky
13,13,Chuy,UY,-33.6971,-53.4616,61.56,55,26,12.21,scattered clouds
16,16,Albany,US,42.6001,-73.9662,67.48,86,100,0.25,overcast clouds
18,18,Atuona,PF,-9.8000,-139.0333,77.56,74,15,17.78,few clouds
19,19,Basco,PH,20.4487,121.9702,84.04,75,96,22.70,overcast clouds
20,20,Malatya,TR,38.5000,38.0000,59.05,45,0,11.50,clear sky


In [5]:
# 4a. Determine if there are any empty rows
preferred_cities_df.count()

City_ID                418
City                   418
Country                414
Lat                    418
Lng                    418
Max Temp               418
Humidity               418
Cloudiness             418
Wind Speed             418
Current Description    418
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy().dropna()
clean_df

,City,Country,Max Temp,Current Description,Lat,Lng
5,Mahebourg,MU,72.25,broken clouds,-20.4081,57.7000
7,Bredasdorp,ZA,50.29,few clouds,-34.5322,20.0403
8,Kalmunai,LK,80.49,overcast clouds,7.4167,81.8167
9,Hilo,US,73.62,light rain,19.7297,-155.0900
12,Nouadhibou,MR,73.38,clear sky,20.9310,-17.0347
...,...,...,...,...,...,...
659,Beyneu,KZ,53.74,overcast clouds,45.3167,55.2000
661,Serik,TR,77.25,clear sky,36.9169,31.0989
663,Beira,MZ,78.89,broken clouds,-19.8436,34.8389
666,Gat,IL,77.68,few clouds,31.6100,34.7642


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Mahebourg,MU,72.25,broken clouds,-20.4081,57.7000,
7,Bredasdorp,ZA,50.29,few clouds,-34.5322,20.0403,
8,Kalmunai,LK,80.49,overcast clouds,7.4167,81.8167,
9,Hilo,US,73.62,light rain,19.7297,-155.0900,
12,Nouadhibou,MR,73.38,clear sky,20.9310,-17.0347,
13,Chuy,UY,61.56,scattered clouds,-33.6971,-53.4616,
16,Albany,US,67.48,overcast clouds,42.6001,-73.9662,
18,Atuona,PF,77.56,few clouds,-9.8000,-139.0333,
19,Basco,PH,84.04,overcast clouds,20.4487,121.9702,
20,Malatya,TR,59.05,clear sky,38.5000,38.0000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City                   414
Country                414
Max Temp               414
Current Description    414
Lat                    414
Lng                    414
Hotel Name             414
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Mahebourg,MU,72.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
7,Bredasdorp,ZA,50.29,few clouds,-34.5322,20.0403,Bredasdorp Country Manor
8,Kalmunai,LK,80.49,overcast clouds,7.4167,81.8167,VS Villa
9,Hilo,US,73.62,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Nouadhibou,MR,73.38,clear sky,20.9310,-17.0347,El Medina
...,...,...,...,...,...,...,...
659,Beyneu,KZ,53.74,overcast clouds,45.3167,55.2000,NUR
661,Serik,TR,77.25,clear sky,36.9169,31.0989,Armas Belek - Ultra All Inclusive
663,Beira,MZ,78.89,broken clouds,-19.8436,34.8389,Beira Terrace Hotel
666,Gat,IL,77.68,few clouds,31.6100,34.7642,OlusHome


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))